In [1]:
import portfolio
import pandas as pd

dt = '2019-12-23'
acct = ('CMS','CITIC')

dfHold = portfolio.get_hold(dt, str(acct))
dfHold

,SymbolCode,SymbolName,Sector,Cur,Qty,CostPrice,Price,MV,PL,Ratio,ExRate,MV_CNY
0,002294.SZ,信立泰,医药,CNY,1000.00,22.457,18.660,18660.00,-3796.74,-0.1691,1.0,18660.000
1,200625.SZ,长安B,汽车,HKD,25200.00,4.582,4.420,111384.00,-4089.03,-0.0354,0.9,100245.600
2,300059.SZ,东方财富,证券,CNY,1400.00,15.537,15.190,21266.00,-486.00,-0.0223,1.0,21266.000
3,510500.SS,500ETF,综合指数,CNY,5000.00,4.870,5.582,27910.00,3560.00,0.1462,1.0,27910.000
4,512880.SS,证券ETF,证券,CNY,58500.00,0.799,0.983,57505.50,10772.50,0.2305,1.0,57505.500
5,600030.SS,中信证券,证券,CNY,1000.00,18.786,23.180,23180.00,4394.00,0.2339,1.0,23180.000
6,600177.SS,雅戈尔,多元企业,CNY,2800.00,5.781,6.840,19152.00,2966.00,0.1832,1.0,19152.000
7,601088.SS,中国神华,煤炭,CNY,2600.00,10.395,17.620,45812.00,18785.00,0.6950,1.0,45812.000
8,601166.SS,兴业银行,银行,CNY,2800.00,15.202,19.900,55720.00,13154.40,0.3090,1.0,55720.000
9,601916.SS,浙商银行,银行,CNY,1000.00,4.940,4.780,4780.00,-160.00,-0.0324,1.0,4780.000


In [11]:
dfSector = dfHold[['Sector','MV_CNY']]
dfSum = dfSector.groupby(['Sector']).sum()
dfSum


,MV_CNY
Sector,
医药,18660.000
多元企业,19152.000
汽车,100245.600
煤炭,45812.000
现金,39822.397
综合指数,27910.000
证券,101951.500
银行,60500.000


In [15]:
total = dfSector[['MV_CNY']].sum()
dfSum['Ratio'] = dfSum.apply(lambda x: round(x['MV_CNY']/total,2),axis=1)
dfSum.loc['合计'] = [float(total), 1]
dfSum

,MV_CNY,Ratio
Sector,,
医药,18660,0.05
多元企业,19152,0.05
汽车,100246,0.24
煤炭,45812,0.11
现金,39822.4,0.10
综合指数,27910,0.07
证券,101952,0.25
银行,60500,0.15
合计,414053,1.00


In [16]:
dfSum.T

Sector,医药,多元企业,汽车,煤炭,现金,综合指数,证券,银行,合计
MV_CNY,18660,19152,100246,45812,39822.4,27910,101952,60500,414053
Ratio,0.05,0.05,0.24,0.11,0.1,0.07,0.25,0.15,1


In [20]:
dfT = dfSum.T
list(dfT.index)

['MV_CNY', 'Ratio']